In [128]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
import psycopg2

# Replace with your own database connection details
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="",
    host="localhost",
    port="5432"
)

# Load transaction data into a Pandas DataFrame
transaksi_df = pd.read_sql_query('SELECT * FROM transactions', conn)
transaksi_df.head()  # Display the first few rows of the transactions table

# Load promo data into a Pandas DataFrame
promo_df = pd.read_sql_query('SELECT * FROM promo', conn)
promo_df.head()  # Display the first few rows of the promo table


/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/3318476716.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transaksi_df = pd.read_sql_query('SELECT * FROM transactions', conn)
/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/3318476716.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  promo_df = pd.read_sql_query('SELECT * FROM promo', conn)


,id,promo_name,category,merchant_name,startdate,enddate
0,1,Dapatkan Potongan Rp100.000 dengan QRIS Wondr ...,6,Grand Lucky,None,2024-12-31
1,2,Dapatkan Penawaran Menarik dengan Kartu Debit ...,1,Aljazeerah Signature,None,2024-12-31
2,3,Dapatkan Bonus Kuota hingga 10 GB,2,Smartfreen,2024-09-06,2024-09-12
3,4,Dapatkan Bonus Goldie 70% dan Diskon hingga Rp...,5,Funworld,2024-07-05,2024-09-30
4,5,wondr by BNI #Jadiinmaumu Tiap Hari Senin,1,Fore,2024-08-05,2024-10-28


**Gradient Boosting Decision Tree + Logistic Regression**

In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Load data
transactions_df = pd.read_sql_query(
    "SELECT id, cif, amount, date, description, category, merchant_name, status_transaction, account_number FROM public.transactions;",
    conn
)

promo_df = pd.read_sql_query(
    "SELECT id, promo_name, category, merchant_name, startdate, enddate FROM public.promo;",
    conn
)

# Merge the data
merged_df = pd.merge(transactions_df, promo_df, on=['merchant_name', 'category'], how='left')

# Create target variable (whether a transaction has a promo or not)
merged_df['has_promo'] = merged_df['promo_name'].notnull().astype(int)

# Ensure all date columns are datetime type
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['startdate'] = pd.to_datetime(merged_df['startdate'])
merged_df['enddate'] = pd.to_datetime(merged_df['enddate'])

# Feature Engineering
merged_df['amount'] = merged_df['amount'].abs()
merged_df['days_since_start'] = (merged_df['date'] - merged_df['startdate']).dt.days.fillna(0)
merged_df['days_until_end'] = (merged_df['enddate'] - merged_df['date']).dt.days.fillna(0)

# Encoding categorical features
le = LabelEncoder()
merged_df['category_encoded'] = le.fit_transform(merged_df['category'])
merged_df['merchant_name_encoded'] = le.fit_transform(merged_df['merchant_name'])

# Features and Target
features = merged_df[['amount', 'days_since_start', 'days_until_end', 'category_encoded', 'merchant_name_encoded']]
target = merged_df['has_promo']


/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/719517462.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions_df = pd.read_sql_query(
/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/719517462.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  promo_df = pd.read_sql_query(


In [114]:
# Preprocess data
def preprocess_data(transactions, promos):
    # Merge transactions and promos on merchant_name and category
    df = pd.merge(transactions, promos, on=['merchant_name', 'category'], how='left')
    
    # Create target variable (1 if promo available, 0 if not)
    df['has_promo'] = df['promo_name'].notna().astype(int)
    
    # Feature engineering
    df['transaction_month'] = pd.to_datetime(df['date']).dt.month
    df['transaction_day'] = pd.to_datetime(df['date']).dt.day
    df['transaction_dayofweek'] = pd.to_datetime(df['date']).dt.dayofweek
    
    # Encode categorical variables
    le = LabelEncoder()
    categorical_cols = ['category', 'merchant_name', 'status_transaction']
    for col in categorical_cols:
        df[col] = le.fit_transform(df[col].astype(str))
    
    # Text features
    tfidf = TfidfVectorizer(max_features=100)
    description_tfidf = tfidf.fit_transform(df['description'].fillna(''))
    description_tfidf = pd.DataFrame(description_tfidf.toarray(), columns=[f'desc_{i}' for i in range(100)])
    
    df = pd.concat([df, description_tfidf], axis=1)
    
    # Select features
    features = ['amount', 'transaction_month', 'transaction_day', 'transaction_dayofweek', 
                'category', 'merchant_name', 'status_transaction'] + [f'desc_{i}' for i in range(100)]
    
    X = df[features]
    y = df['has_promo']
    
    return X, y

X, y = preprocess_data(transactions, promos)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [127]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

# Define base models
gb_model = GradientBoostingClassifier()
lr_model = LogisticRegression(max_iter=500)

# Create a stacked classifier
stacked_model = StackingClassifier(
    estimators=[('gb', gb_model)],
    final_estimator=lr_model
)

# Define the pipeline with StackingClassifier
pipeline = Pipeline([
    ('stacked', stacked_model)
])

# Define hyperparameters for tuning
param_grid = {
    'stacked__gb__n_estimators': [100, 200],
    'stacked__gb__learning_rate': [0.1, 0.05],
    'stacked__gb__max_depth': [3, 5],
    'stacked__final_estimator__C': [0.1, 1.0, 10.0]
}

# Hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predicting on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report: \n{report}")


Best Parameters: {'stacked__final_estimator__C': 0.1, 'stacked__gb__learning_rate': 0.1, 'stacked__gb__max_depth': 3, 'stacked__gb__n_estimators': 100}
Accuracy: 1.0
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       814
           1       1.00      1.00      1.00      5858

    accuracy                           1.00      6672
   macro avg       1.00      1.00      1.00      6672
weighted avg       1.00      1.00      1.00      6672



In [122]:
# Get prediction probabilities
y_pred_proba = best_model.predict_proba(X_test)

In [126]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'best_model' has already been trained and 'X_test', 'y_test' are available

# Get prediction probabilities for each class
y_pred_proba = best_model.predict_proba(X_test)

# Function to calculate Average Precision at k
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# Function to calculate Mean Average Precision at k
def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

# Generate the true labels in the form of a list of lists
actual = [[label] for label in y_test]

# Generate predictions: ranked indices based on the predicted probabilities
predicted = [np.argsort(proba)[::-1][:k] for proba in y_pred_proba]  # Get the top-k predictions

# Set the value of k
k = 10

# Calculate MAP@k
mapk_score = mapk(actual, predicted, k=k)

print(f'MAP@{k}: {mapk_score}')


MAP@10: 1.0


**RNN**

**Light GBM**

In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Load data
transactions = pd.read_sql("SELECT id, cif, amount, date, description, category, merchant_name, status_transaction, account_number FROM public.transactions", conn)
promos = pd.read_sql("SELECT id, promo_name, category, merchant_name, startdate, enddate FROM public.promo", conn)

# Preprocess data
le_merchant = LabelEncoder()
le_category = LabelEncoder()

transactions['merchant_id'] = le_merchant.fit_transform(transactions['merchant_name'])
transactions['category_id'] = le_category.fit_transform(transactions['category'])

promos['merchant_id'] = le_merchant.transform(promos['merchant_name'])
promos['category_id'] = le_category.transform(promos['category'])

# Create features
X = transactions[['amount', 'merchant_id', 'category_id']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Cluster the scaled transactions directly
n_clusters = 10  # Adjust this as needed
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# Function to get recommendations
def get_recommendations(user_transactions, n_recommendations=10):
    user_clusters = cluster_labels[user_transactions.index]
    cluster_counts = np.bincount(user_clusters)
    dominant_cluster = np.argmax(cluster_counts)
    
    # Find promos in the same category or merchant as the dominant cluster
    cluster_transactions = transactions[cluster_labels == dominant_cluster]
    relevant_categories = set(cluster_transactions['category_id'])
    relevant_merchants = set(cluster_transactions['merchant_id'])
    
    relevant_promos = promos[
        (promos['category_id'].isin(relevant_categories)) | 
        (promos['merchant_id'].isin(relevant_merchants))
    ]
    
    # If we have more relevant promos than requested, sample randomly
    if len(relevant_promos) > n_recommendations:
        return relevant_promos.sample(n_recommendations)
    else:
        return relevant_promos

# Evaluation functions
def precision_at_k(recommended_items, relevant_items, k):
    recommended_items = set(recommended_items[:k])
    relevant_items = set(relevant_items)
    return len(recommended_items.intersection(relevant_items)) / k

def recall_at_k(recommended_items, relevant_items, k):
    recommended_items = set(recommended_items[:k])
    relevant_items = set(relevant_items)
    return len(recommended_items.intersection(relevant_items)) / len(relevant_items)

def f1_at_k(recommended_items, relevant_items, k):
    p = precision_at_k(recommended_items, relevant_items, k)
    r = recall_at_k(recommended_items, relevant_items, k)
    return 2 * (p * r) / (p + r) if (p + r) > 0 else 0

def mean_average_precision(recommended_items, relevant_items):
    ap = 0
    hits = 0
    for i, item in enumerate(recommended_items):
        if item in relevant_items:
            hits += 1
            ap += hits / (i + 1)
    return ap / len(relevant_items)

def mean_reciprocal_rank(recommended_items, relevant_items):
    if len(relevant_items) == 0:
        return 0
    for rank, item in enumerate(recommended_items, start=1):
        if item in relevant_items:
            return 1 / rank
    return 0

# Evaluate the model
def evaluate_model(n_recommendations=20, n_users=1000):
    precisions, recalls, f1_scores, maps, mrrs = [], [], [], [], []
    
    for user in transactions['cif'].unique()[:n_users]:
        user_transactions = transactions[transactions['cif'] == user]
        recommendations = get_recommendations(user_transactions, n_recommendations)
        
        relevant_items = set(user_transactions['merchant_id']).union(set(user_transactions['category_id']))
        recommended_items = list(recommendations['merchant_id']) + list(recommendations['category_id'])
        
        precisions.append(precision_at_k(recommended_items, relevant_items, n_recommendations))
        recalls.append(recall_at_k(recommended_items, relevant_items, n_recommendations))
        f1_scores.append(f1_at_k(recommended_items, relevant_items, n_recommendations))
        maps.append(mean_average_precision(recommended_items, relevant_items))
        mrrs.append(mean_reciprocal_rank(recommended_items, relevant_items))
    
    print(f"Average Precision@{n_recommendations}: {np.mean(precisions)}")
    print(f"Average Recall@{n_recommendations}: {np.mean(recalls)}")
    print(f"Average F1@{n_recommendations}: {np.mean(f1_scores)}")
    print(f"Mean Average Precision: {np.mean(maps)}")
    print(f"Mean Reciprocal Rank: {np.mean(mrrs)}")

# Run evaluation
evaluate_model()


Average Precision@20: 1.0
Average Recall@20: 0.2621326042378674
Average F1@20: 0.4153780068728522
Mean Average Precision: 0.5242652084757348
Mean Reciprocal Rank: 1.0


/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/1616642664.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions = pd.read_sql("SELECT id, cif, amount, date, description, category, merchant_name, status_transaction, account_number FROM public.transactions", conn)
/var/folders/hh/yvtjm3s173l6bkfz3ydwcj9023vg_6/T/ipykernel_87054/1616642664.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  promos = pd.read_sql("SELECT id, promo_name, category, merchant_name, startdate, enddate FROM public.promo", conn)


**Tuning Light GBM**

In [95]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Define a scorer for KMeans (precision or other relevant metrics)
def f1_scorer(estimator, X):
    cluster_labels = estimator.predict(X)
    return np.mean([
        f1_at_k(
            get_recommendations(transactions[transactions['cif'] == user], 5)['merchant_id'].values,
            set(transactions[transactions['cif'] == user]['merchant_id']),
            5
        )
        for user in transactions['cif'].unique()
    ])

f1_scorer_fn = make_scorer(f1_scorer, greater_is_better=True)

# Define hyperparameters to tune
param_grid = {
    'n_clusters': [5, 10, 15],  # Number of clusters
    'init': ['k-means++', 'random'],  # Initialization method
    'max_iter': [100, 300],  # Max iterations
    'n_init': [10, 20]  # Number of runs with different seeds
}

# Initialize KMeans
kmeans = KMeans(random_state=42)

# Grid Search CV for finding the best parameters
grid_search = GridSearchCV(kmeans, param_grid, scoring=f1_scorer_fn, cv=3, verbose=1)
grid_search.fit(X_scaled)

# Get best parameters
print("Best parameters found:", grid_search.best_params_)

# Use the best parameters
best_kmeans = grid_search.best_estimator_

# Cluster with the best KMeans
best_cluster_labels = best_kmeans.fit_predict(X_scaled)

# Update the cluster labels for future recommendation logic
cluster_labels = best_cluster_labels

# Re-run evaluation
evaluate_model()


Fitting 3 folds for each of 24 candidates, totalling 72 fits


/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 969, in _score
    scores = scorer(estimator, X_test, **score_params)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 969, in _score
    scores = scorer(estimator, X_test, **score_params)
TypeError: _BaseScorer.__ca

Best parameters found: {'init': 'k-means++', 'max_iter': 100, 'n_clusters': 5, 'n_init': 10}
Average Precision@5: 1.0
Average Recall@5: 0.06553315105946685
Average F1@5: 0.1230051189400783
Mean Average Precision: 0.1310663021189337
